In [1]:
import os
import sys
import time as time

os.environ["CUDA_VISIBLE_DEVICES"] = "4"

# 获取当前文件的绝对路径
current_file_path = os.getcwd()

# 跳出 g_unet 目录，进入 Xinyi_GNN_aorta/src
project_root = os.path.abspath(os.path.join(current_file_path, ".."))
src_path = os.path.join(project_root, "src")

print(f"src path: {src_path}")
sys.path.insert(0, src_path)

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

src path: /ehome/xinyi/Xinyi_GNN_aorta/src


device(type='cuda')

In [2]:
# from utils import aorta_3D_info, Normalizer_ts,vtp2Graph_aorta

# # In this code block we will derive the data information for the aorta_toy dataset.
# raw_data_path = '/ehome/xinyi/Xinyi_GNN_aorta/data/toy_unet/raw'
# outputpath = '/ehome/xinyi/Xinyi_GNN_aorta/data/toy_unet'
# data_label = ['y', 'pos', 'stmdist']
# data_information = aorta_3D_info(path=raw_data_path, labels=data_label, output_path=outputpath)

In [3]:
from utils import suk_aorta_3D_info, Normalizer_ts

processed_path = '/ehome/xinyi/Xinyi_GNN_aorta/data/vessel-dataset/single_toy_unet/raw'
outputpath     = '/ehome/xinyi/Xinyi_GNN_aorta/data/vessel-dataset/single_toy_unet'
data_label = ['y', 'pos', "stmdist"]

data_information = suk_aorta_3D_info(path=processed_path, labels=data_label, output_path=outputpath)
dfs  = data_information.graph2pandas()


csv already exists, loading data...
data load done.


In [4]:
data_information.info

,max,min,mean,std
label,,,,
pressure,139176.950000,131830.520000,134748.730000,1287.432600
wssx,403.141170,-134.361650,21.726862,29.298239
wssy,225.556690,-224.741970,0.424268,20.644999
wssz,90.428570,-100.762505,-0.004054,5.544290
x,6.126948,-1.918083,2.098806,2.182891
y,1.982128,-2.325242,-0.062998,0.609867
z,0.198804,-0.198806,0.000279,0.119057
distance,8.815603,-0.041465,3.953391,2.390356


In [5]:
# import numpy as np

# # Extracting output and input ranges from the data information for data normalization
# output_range = data_information.info.values[1].reshape(1, -1) # pressure!!!!!!!!!!!!!
# input_range = data_information.info.values[4:] 

# output_range = output_range.astype(np.float32)
# input_range = input_range.astype(np.float32)

In [6]:
import numpy as np

# Extracting output ranges from the data information for data normalization

output_range = data_information.info.values[0].reshape(1, -1) # pressure!!!!!!!!!!!!!
output_range = output_range.astype(np.float32)

# Step 1: 提取 x, y, z 三行的数据（索引为 4, 5, 6）
xyz_rows = data_information.info.values[4:7].astype(np.float32)

# Step 2: 提取 mean 和 std 列（索引 2 和 3）
xyz_mean = xyz_rows[:, 2]  # shape = (3,)
xyz_std = xyz_rows[:, 3]   # shape = (3,)

# Step 3: 计算 mean 和 std 的平均值
shared_mean = np.mean(xyz_mean)
shared_std = np.mean(xyz_std)

# Step 4: 构造新的 mean/std 参数，用于 x/y/z 和其他维度（比如 distance）拼接
# x/y/z 使用统一的 mean/std，distance 用原来的
distance_row = data_information.info.values[7].astype(np.float32)  # index 7 是 distance

# 构造 input normalization 参数，按列顺序排列
# shape = (2, 4) → [mean_row, std_row]
input_mean = np.array([shared_mean, shared_mean, shared_mean, distance_row[2]])
input_std = np.array([shared_std, shared_std, shared_std, distance_row[3]])
input_params = np.vstack([input_mean, input_std])  # shape = (2, 4)

In [ ]:
# input_normalizer = Normalizer_ts(method='ms',params=input_range[:,2:].T)
input_normalizer = Normalizer_ts(method='ms', params=input_params)
output_normalizer = Normalizer_ts(method="ms", params=output_range[:,2:].T)

In [8]:
from Bench_Heat.heat_sampling_suk import HeatSamplingCluster
from dataset import Aorta_3d_Dataset

# Multiscale radius graph
ratios = [1., 0.4, 0.1]
radii = [0.042, 0.06, 0.8] # if artery_type == 'single' 
# else [0.022, 0.04, 0.1]

trans = HeatSamplingCluster(ratios, radii)


root = '/ehome/xinyi/Xinyi_GNN_aorta/data/vessel-dataset/single_toy_unet'
dataset = Aorta_3d_Dataset(root,'cpu', label='pressure', pre_transform=trans,input_normalizer=input_normalizer,output_normalizer=output_normalizer)

Processing...
100%|██████████| 11/11 [00:08<00:00,  1.25it/s]
Done!


In [9]:
# import os
# import numpy as np
# import torch
# import pyvista as pv

# def export_multiscale_vtp(data, prefix='aorta'):
#     """
#     兼容 MultiscaleData:
#     x=[N, *], pos=[N, 3], face=[3, F],
#     scale{i}_sample_index, scale{i}_cluster_map, scale{i}_edge_index, ...
#     导出：
#       1) {prefix}_multiscale_mesh.vtp  （带每层 mask 与 cluster）
#       2) {prefix}_scale{i}_points.vtp  （各层采样点云）
#       3) {prefix}_multiscale_bundle.vtm（MultiBlock，方便一次性打开）
#     """
#     # -------- 0) 取点坐标（pos_raw优先）
#     points = data.pos_raw.cpu().numpy() if hasattr(data, 'pos_raw') else data.pos.cpu().numpy()
#     N = points.shape[0]

#     # -------- 1) 构造 faces（若有）
#     faces = None
#     if hasattr(data, 'face') and (data.face is not None):
#         face_t = data.face
#         if isinstance(face_t, torch.Tensor):
#             face_np = face_t.t().contiguous().cpu().numpy()  # (F,3)
#         else:
#             face_np = np.asarray(face_t)
#         if face_np.size > 0:
#             F = face_np.shape[0]
#             faces = np.hstack([np.full((F, 1), 3, dtype=np.int64), face_np]).ravel().astype(np.int64)

#     # -------- 2) 基础网格
#     mesh = pv.PolyData(points, faces) if faces is not None else pv.PolyData(points)

#     # -------- 3) 找出可用的 scale 键（兼容 sample_index / sampling_index）
#     keys = list(data.keys())
#     scale_ids = []
#     for k in keys:
#         if k.startswith('scale') and (k.endswith('sample_index') or k.endswith('sampling_index')):
#             # 提取数字 id
#             sid = int(k[len('scale'):].split('_')[0])
#             scale_ids.append(sid)
#     scale_ids = sorted(set(scale_ids))

#     print("Detected scales:", scale_ids)

#     # MultiBlock 容器（把完整网格 + 各层点云一起打包）
#     mb = pv.MultiBlock()
#     mb['full_mesh'] = mesh.copy()

#     # 保存一个全局的原始点序号（调试/显示用）
#     mesh.point_data['point_id'] = np.arange(N, dtype=np.int32)

#     # -------- 4) 逐层写 mask / cluster，并导出各层点云
#     for sid in scale_ids:
#         # 取 index 键名（优先 sample_index）
#         idx_key1 = f'scale{sid}_sample_index'
#         idx_key2 = f'scale{sid}_sampling_index'
#         if hasattr(data, idx_key1):
#             idx_t = getattr(data, idx_key1)
#         else:
#             idx_t = getattr(data, idx_key2)

#         # 取 cluster（若有）
#         cluster_key = f'scale{sid}_cluster_map'
#         cluster_arr = None
#         if hasattr(data, cluster_key):
#             cluster_t = getattr(data, cluster_key)
#             cluster_arr = cluster_t.detach().cpu().numpy().astype(np.int32).ravel()
#             if cluster_arr.shape[0] == N:
#                 mesh.point_data[f'scale{sid}_cluster'] = cluster_arr
#             else:
#                 # 长度不等于 N 就不写到 full mesh 上，避免误导
#                 pass

#         # 采样索引清洗
#         idxs = idx_t.detach().cpu().numpy().astype(np.int64).ravel()
#         # 有些实现可能返回布尔/全量映射，这里统一转“唯一且在界内的点索引”
#         if idxs.dtype == np.bool_ or ((idxs.ndim == 1) and np.array_equal(np.unique(idxs), np.array([0,1])) and idxs.size==N):
#             # 万一是布尔 mask（极少见），转成索引
#             idxs = np.where(idxs.astype(bool))[0]
#         else:
#             idxs = np.unique(idxs)
#             idxs = idxs[(idxs >= 0) & (idxs < N)]

#         # 写 mask（0/1），ParaView 里用 Threshold 选 0.5~1.5 即可筛到采样点
#         mask = np.zeros((N,), dtype=np.uint8)
#         mask[idxs] = 1
#         mesh.point_data[f'scale{sid}_mask'] = mask

#         # 导出该层点云
#         sampled_pts = points[idxs] if idxs.size > 0 else points[:0]
#         sampled_cloud = pv.PolyData(sampled_pts)
#         sampled_cloud.point_data['ones'] = np.ones((sampled_pts.shape[0],), dtype=np.float32) if idxs.size > 0 else np.array([], dtype=np.float32)

#         pts_path = f'{prefix}_scale{sid}_points.vtp'
#         sampled_cloud.save(pts_path, binary=True)
#         print(f"[scale{sid}] sampled points: {pts_path}  (count={sampled_pts.shape[0]})")

#         # MultiBlock 增加一块
#         mb[f'scale{sid}_points'] = sampled_cloud

#     # -------- 5) 保存带多尺度属性的整网格
#     mesh_path = f'{prefix}_multiscale_mesh.vtp'
#     mesh.save(mesh_path, binary=True)
#     print(f"Saved mesh with masks/clusters: {mesh_path}")

#     # -------- 6) 保存 MultiBlock
#     bundle_path = f'{prefix}_multiscale_bundle.vtm'
#     mb.save(bundle_path)
#     print(f"Saved multiscale bundle: {bundle_path}")

# # ====== 用法：取一个样本然后导出 ======
# data = dataset[0]  # 你的 MultiscaleData
# export_multiscale_vtp(data, prefix='aorta')


In [10]:

raw_data = torch.load('/ehome/xinyi/Xinyi_GNN_aorta/data/vessel-dataset/single_toy_unet/processed/data_0.pt', weights_only=False)
print(raw_data.keys())  # 应该至少有 'edge_index'


['scale0_cluster_map', 'y', 'scale0_edge_index', 'scale2_sample_index', 'scale1_cluster_map', 'scale0_sample_index', 'scale2_cluster_map', 'face', 'scale2_edge_index', 'norm', 'scale1_edge_index', 'pos', 'x', 'scale1_sample_index']


In [11]:
print(dataset[0])

data = dataset[0].to(device)
mean = data.y.mean()
std = data.y.std()

# print(f"Mean: {mean.item():.4f}, Std: {std.item():.4f}")


MultiscaleData(x=[10475, 4], y=[10475], pos=[10475, 3], norm=[10475, 3], face=[3, 20946], scale0_cluster_map=[10475], scale0_edge_index=[2, 75136], scale2_sample_index=[320], scale1_cluster_map=[10475], scale0_sample_index=[10475], scale2_cluster_map=[3493], scale2_edge_index=[2, 10198], scale1_edge_index=[2, 13048], scale1_sample_index=[3493])


In [12]:
# from torch_geometric.loader import DataLoader
# from torch.utils.data import random_split

# train_size = 10
# valid_size = 1
# test_size = 1
# train_dataset, test_dataset= random_split(dataset, [train_size, test_size], generator=torch.Generator().manual_seed(0))
# _, valid_dataset = random_split(train_dataset, [train_size-valid_size, valid_size], generator=torch.Generator().manual_seed(1))

# train_loader = DataLoader(train_dataset, batch_size = 6, shuffle=False)
# valid_loader = DataLoader(valid_dataset, batch_size = 1)
# test_loader = DataLoader(test_dataset, batch_size = 1)

In [13]:
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split

total = len(dataset)
train_size = int(0.9 * total)
valid_size = int(0.1 * total)
test_size  = total - train_size

train_dataset, test_dataset= random_split(dataset, [train_size, test_size], generator=torch.Generator().manual_seed(0))
_, valid_dataset = random_split(train_dataset, [train_size-valid_size, valid_size], generator=torch.Generator().manual_seed(1))

train_loader = DataLoader(train_dataset, batch_size = 4, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size = 1)
test_loader = DataLoader(test_dataset, batch_size = 1)

In [14]:
# # Create an iterator from the loader
# train_iterator = iter(train_loader)

# # Get the first batch
# first_batch = next(train_iterator)
# print("First batch:")
# print(first_batch)


# # Get the second batch
# second_batch = next(train_iterator)
# print("\nSecond batch:")
# print(second_batch)

# # ... and so on until the iterator is exhausted.

In [15]:
from Bench_Heat.baseline import AttGCN
from torch.optim.lr_scheduler import ExponentialLR

model = AttGCN()
model.to('cuda')


lr = 0.001

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=lr,weight_decay=0.001)
schedule = ExponentialLR(optimizer,.999)

optim_para = {
            'epoch': 0,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss':1,
            }

AttGCN (2225861 trainable parameters)


In [16]:
# model

In [17]:
# from tqdm import tqdm
# from traincopy import train, valid  # only pressure!!! 后续记得baseline改回3维

# import matplotlib.pyplot as plt
# #start training 
# train_epoch_error = []
# val_epoch_error = []
# test_error = []
# check_idx = []
# epochs = 1000

# for epoch in tqdm(range(epochs)):
#     train_epoch_error.append(train(model, device, train_loader, optimizer,criterion,scheduler=schedule))
#     val_epoch_error.append(valid(model, device, valid_loader,criterion))
#     if (epoch+1)%100==0 or epoch==0:
#         # val_epoch_error.append(valid(model, device, valid_loader,criterion))
#         test_error.append(valid(model, device, test_loader,criterion))
#         check_idx.append(epoch)
#     # if optim_para['loss'] > train_epoch_error[epoch]:
#     #     optim_para['epoch'] = epoch
#     #     optim_para['model_state_dict'] = model.state_dict()
#     #     optim_para['optimizer_state_dict']= optimizer.state_dict()
#     #     optim_para['loss'] = train_epoch_error[epoch]
#     # check_idx.append(epoch+1)


#     if (epoch+1)%200==0 and epoch != 0:    
#         fig, ax = plt.subplots(figsize=(10,8))
#         er1 = torch.stack(train_epoch_error)
#         er2 = torch.stack(val_epoch_error)
#         er3 = torch.stack(test_error)
#         ax.plot(er1.detach().cpu(),color = 'C0',linestyle='solid',linewidth=1,alpha=1,label='L_train')
#         ax.plot(er2.detach().cpu(),color = 'C1',linestyle='solid',linewidth=1,alpha=1,label='L_valid')
#         ax.plot(check_idx,er3.detach().cpu(),color = 'C2',linestyle='solid',linewidth=1,alpha=1,label='L_test')
#         ax.set_yscale('log')
#         ax.set_title('train_loss_curve')
#         ax.set_xlabel('epochs')
#         ax.set_ylabel('loss')
#         ax.legend()
#         plt.show()

In [18]:
from tqdm import tqdm
from train import train, valid

import torch
import matplotlib.pyplot as plt
import os

# ✅ checkpoint 路径
ckpt_dir = 'checkpoints_suk_toy_no_norm'
os.makedirs(ckpt_dir, exist_ok=True)

# ✅ 图像保存路径
fig_dir = 'figures_suk_toy_no_norm'
os.makedirs(fig_dir, exist_ok=True)

# ✅ 初始化日志
train_epoch_error = []
val_epoch_error = []
test_epoch_error = []
check_idx = []

epochs = 1000

# ✅ 开始训练
for epoch in tqdm(range(epochs)):
    train_loss = train(model, device, train_loader, optimizer, criterion, scheduler=schedule)
    train_epoch_error.append(train_loss)

    if (epoch  % 100) == 0:
        val_loss = valid(model, device, valid_loader, criterion)
        test_loss = valid(model, device, test_loader, criterion)

        val_epoch_error.append(val_loss)
        test_epoch_error.append(test_loss)
        check_idx.append(epoch)

        # ✅ 保存 checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss,
            'test_loss': test_loss
        }, os.path.join(ckpt_dir, f'checkpoint_epoch_{epoch}.pt'))

        # ✅ 画图 1：训练损失
        fig1, ax1 = plt.subplots(figsize=(8, 5))
        ax1.plot(
            range(len(train_epoch_error)),
            torch.stack(train_epoch_error).detach().cpu().numpy(),
            color='C0', label='Train Loss'
        )
        ax1.set_yscale('log')
        ax1.set_title('Train Loss Curve')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax1.legend()
        fig1.tight_layout()
        fig1.savefig(os.path.join(fig_dir, f"train_loss_epoch_{epoch}.png"))
        plt.close(fig1)

        # ✅ 画图 2：Val/Test 损失（每 100 epoch）
        fig2, ax2 = plt.subplots(figsize=(8, 5))
        ax2.plot(
            check_idx,
            torch.stack(val_epoch_error).detach().cpu().numpy(),
            color='C1', label='Validation Loss'
        )
        ax2.plot(
            check_idx,
            torch.stack(test_epoch_error).detach().cpu().numpy(),
            color='C2', label='Test Loss'
        )
        ax2.set_yscale('log')
        ax2.set_title('Validation & Test Loss (every 100 epochs)')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Loss')
        ax2.legend()
        fig2.tight_layout()
        fig2.savefig(os.path.join(fig_dir, f"val_test_loss_epoch_{epoch}.png"))
        plt.close(fig2)


100%|██████████| 1000/1000 [11:23<00:00,  1.46it/s]


In [19]:
# import torch
# import pyvista as pv
# import matplotlib.pyplot as plt
# from os.path import join as osj
# from tqdm import tqdm


# def generate_output(
#     model,
#     device,
#     output_loader,
#     raw_mesh_path,
#     tag,
#     save_mesh_path,
#     mesh_name="mesh",
#     output_normalizer=None,
# ):
#     for _, batch in enumerate(tqdm(output_loader)):
#         batch = batch.to(device)
#         with torch.no_grad():
#             label = batch.y
#             output = model(batch)
#             label_denorm = output_normalizer.denormalize(label)  # shape is <N,1>
#             output_denorm = output_normalizer.denormalize(output)  # shape is <N,1>
#             idx = (batch.idx)[0]

#             mesh = pv.read(osj(raw_mesh_path, "aorta00{:d}.vtp".format(idx)))
#             mesh.point_data[tag + "_prediction"] = output_denorm.detach().cpu()
#             mesh.point_data[tag + "_label"] = label_denorm.detach().cpu()
#             mesh.point_data[tag + "_error"] = (
#                 mesh.point_data[tag + "_prediction"] - mesh.point_data[tag + "_label"]
#             )
#             mesh.save(osj(save_mesh_path, mesh_name + "_{:d}.vtp".format(idx)))
# generate_output(model_opt,'cpu',valid_loader,'aorta/raw','wss','results/meshes','mesh_wss_50',output_normalizer)